In [1]:
import nltk
import pandas as pd
from happytransformer import HappyTextClassification
from pathlib import Path
from newsapi import NewsApiClient
from sentiment_analisys import get_digests_by_terms_async, classify_digests_async, Prediction

In [2]:
nltk.data.path = [str(Path().resolve().parent.joinpath("nltk_data"))]

# nltk.download("punkt", download_dir="./nltk_data")

# Init pass 9gPj8KZ8XJsLbcT
api = NewsApiClient(api_key='98d763c2280d4820913c4abf3ff0270b')

# classifier = HappyTextClassification(
#     model_type="DISTILBERT", num_labels=2,
#     model_name="distilbert-base-uncased-finetuned-sst-2-english")

classifier = HappyTextClassification(
    model_type="BERT", num_labels=3,
    model_name="ProsusAI/finbert")

07/05/2022 01:57:53 - INFO - happytransformer.happy_transformer -   Using model: cpu


In [3]:
# [source["id"] for source in api.get_sources()["sources"]]

In [4]:

# sources = "business-insider,crypto-coins-news,financial-post,fortune,google-news,info-money,hacker-news,reddit-r-all,Bloomberg"
sources = "business-insider,crypto-coins-news,financial-post,fortune,google-news,info-money,hacker-news,reddit-r-all"

terms = "bitcoin,ethereum"

digests_by_term = await get_digests_by_terms_async(terms, sources, api)

predictions: [Prediction] = [
    Prediction(
        results=await classify_digests_async(digests, classifier),
        term=term
    ) for term, digests in digests_by_term
]

# [digest.summary for digest in predictions[0]["digests"]]

# [prediction for prediction in predictions]

In [5]:
values = [(prediction.term, classification.label, classification.score, digest.url)
          for prediction in predictions
          for (digest, classification) in prediction.results]

df = pd.DataFrame(values, columns= ['term','label', 'score', 'source'])

df

,term,label,score,source
0,bitcoin,negative,0.558181,https://www.businessinsider.com/british-army-t...
1,bitcoin,neutral,0.893477,https://www.businessinsider.com/senator-cynthi...
2,bitcoin,positive,0.637387,https://fortune.com/2022/07/04/meitu-net-loss-...
3,bitcoin,negative,0.730556,https://markets.businessinsider.com/news/curre...
4,ethereum,negative,0.558181,https://www.businessinsider.com/british-army-t...


In [6]:
for prediction in predictions:
    term = prediction.term
    labels = {"negative": 0, "positive": 0, "neutral": 0}
    results_count = len(prediction.results)
    for digest, classification in prediction.results:
        labels[classification.label] += 1
        total = labels["positive"] + labels["negative"] + labels["neutral"]
        # print(f"{term} = {classification.label}")
        if results_count == total:
            positive_ratio = labels["positive"] / total
            negative_ratio = labels["negative"] / total
            neutral_ratio = labels["neutral"] / total
            should_buy = positive_ratio > negative_ratio and positive_ratio > neutral_ratio
            print(f"{term}: {'should not buy' if should_buy else 'should not buy'}")
            print(f"positive = {positive_ratio}")
            print(f"negative = {negative_ratio}")
            print(f"neutral = {neutral_ratio}")
            print("")


bitcoin: should not buy
positive = 0.25
negative = 0.5
neutral = 0.25

ethereum: should not buy
positive = 0.0
negative = 1.0
neutral = 0.0

